In [ ]:
# Exploration notebook for Advanced Personal Finance Tracker
# This notebook is for prototyping data analysis and visualization

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import src.database as db

# Fetch transactions from database
rows = db.get_transactions()
columns = ["txn_id", "user_id", "date", "amount", "type", "category_id", "description", "tags"]
df = pd.DataFrame(rows, columns=columns)

# Display basic info
print("Data shape:", df.shape)
print(df.head())

# Convert date to datetime type
df['date'] = pd.to_datetime(df['date'])

# Basic statistics
print("\nSummary statistics for amount:")
print(df['amount'].describe())

# Count transactions per type
print("\nTransaction type counts:")
print(df['type'].value_counts())

# Monthly aggregation of income and expenses
df['year_month'] = df['date'].dt.to_period('M')
monthly_agg = df.groupby(['year_month', 'type'])['amount'].sum().unstack(fill_value=0)
monthly_agg['net_savings'] = monthly_agg.get('income', 0) - monthly_agg.get('expense', 0)

print("\nMonthly aggregation:")
print(monthly_agg)

# Plot monthly income vs expenses
plt.figure(figsize=(12,6))
monthly_agg[['income','expense']].plot(kind='bar')
plt.title("Monthly Income and Expenses")
plt.xlabel("Year-Month")
plt.ylabel("Amount")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Category-wise spending
categories = db.execute_query("SELECT category_id, name FROM categories")
cat_map = {cid: name for cid, name in categories} if categories else {}
df['category_name'] = df['category_id'].map(cat_map)

expense_df = df[df['type'] == 'expense']
category_spending = expense_df.groupby('category_name')['amount'].sum().sort_values(ascending=False)

print("\nSpending by Category:")
print(category_spending)

# Plot category spending pie chart
plt.figure(figsize=(8,8))
category_spending.plot.pie(autopct='%1.1f%%', startangle=140)
plt.ylabel('')
plt.title("Expense Distribution by Category")
plt.show()
